# Bag of Words : 각 단어의 빈도로 만들어진 백터

In [1]:
# using keras tokenizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# BoW

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence)

bow = dict(tokenizer.word_counts)

print('Bag of Words :', bow)
print('vocab size :', len(tokenizer.index_word))

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
vocab size : 10


In [3]:
#using sklearn CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words :', bow)
print('vocab size : ', len(vector.vocabulary_))

Bag of Words : [[1 1 1 1 3 2 2 2 1 2]]
vocab size :  10


In [4]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [5]:
print('{:.2f}'.format(cos_sim(doc1, doc2))) #문서1과 문서2의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc1, doc3))) #문서1과 문서3의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc2, doc3))) #문서2과 문서3의 코사인 유사도

0.67
0.67
1.00


In [6]:
# Document-Term Matrix using sklearn

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',    
]

vector = CountVectorizer()
dtm = vector.fit_transform(corpus).toarray()

print('dtm :',dtm)
print('vocabulary size :', len(vector.vocabulary_))

dtm : [[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
vocabulary size : 10


#### DTM 의 2가지 한계점

1. DTM은 문서-단어 수가 많아질수록, 행렬은 0의 값을 가진 차원이 증가하게 됩니다. 이는 메모리 측면에서 공간낭비입니다.
2. DTM은 단어의 빈도에만 집중하게 된다는 한계가 있습니다. 이에 따라 빈도가 많이 나타나는 조사(the)나 중요하지 않은 단어들이 문서의 특징이 되어버립니다. 그래서 빈도가 높은 단어, 특히 중요하지 않은 단어,들 중심으로 문서들이 분류가 될 수 있는 문제가 발생할 수 있습니다. 
3. 단어의 의미를 백터화하지 못함

# TF-IDF : 각 단어의 중요도를 판단하여 가중치를 주는 방법

In [7]:
from math import log
import pandas as pd
print('=>')

=>


In [8]:
docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]
print('=3')

=3


In [9]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('vocab size : ', len(vocab))
print(vocab)

vocab size :  13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [10]:
N = len(docs)
print(N)

3


In [11]:
def tf(t,d):
    return d.count(t)

def idf(t) :
    df = 0
    for doc in docs :
        df += t in doc
    return log(N/(df+1))+1

def tfidf(t,d) :
    return tf(t,d) * idf(t)

In [12]:
result = []

for i in range(N) :
    result.append([])
    d = docs[i]
    for j in range(len(vocab)) :
        t = vocab[j]
        
        result[-1].append(tf(t,d))
        
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [13]:
result = []

for j in range(len(vocab)) :
    t = vocab[j]
    result.append(idf(t))
    
idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


In [14]:
result = []

for i in range(N) :
    result.append([])
    d = docs[i]
    for j in range(len(vocab)) :
        t = vocab[j]
        
        result[-1].append(tfidf(t,d))
        
tfidf_ = pd.DataFrame(result, columns =vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


In [15]:
# sklearn을 이용한 TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(docs)
vocab = list(tfidfv.vocabulary_.keys())
vocab.sort()

tfidfv_ = pd.DataFrame(tfidfv.transform(docs).toarray(), columns=vocab)
tfidfv_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


# LSA와 LDA

### LSA(Latent Semantic Analysis) : 문서 속 단어들 사이의 관계를 찾자

In [16]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
print('=3')

=3


In [17]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /aiffel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
import os

csv_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/abcnews-date-text.csv'

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

('/aiffel/aiffel/topic_modelling/data/abcnews-date-text.csv',
 <http.client.HTTPMessage at 0x7f134750e610>)

In [19]:
data = pd.read_csv(csv_filename, on_bad_lines='skip')
data.shape

(1082168, 2)

In [20]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [21]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [22]:
text.nunique()

headline_text    1054983
dtype: int64

In [23]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

In [24]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [25]:
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


In [26]:
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc
print('=3')

=3


In [27]:
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [28]:
# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)
print('=3')

=3


In [29]:
print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


In [30]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.20389733e-02, -3.68541466e-03,  1.83272694e-02, ...,
         3.23326213e-03,  3.24959996e-03,  1.15531954e-02],
       [ 2.90340328e-02, -1.09169757e-02,  1.82508757e-02, ...,
        -2.26542105e-03, -1.05628343e-02, -6.20760272e-03],
       [ 5.03537552e-03, -1.99797818e-03,  9.77760444e-03, ...,
        -1.95179404e-03,  2.70682454e-03,  1.20248715e-03],
       ...,
       [ 2.96933751e-02,  3.91113017e-03,  2.56156108e-02, ...,
         3.08953169e-02,  1.16007189e-02, -5.40909558e-03],
       [ 6.17141709e-02, -5.38431930e-03,  1.36402601e-01, ...,
         9.36967522e-01,  7.71270857e-01, -2.49782034e-01],
       [ 7.13273662e-02,  2.96330020e-02, -6.94105995e-04, ...,
         1.70136800e-02,  2.46636446e-02,  4.85302398e-03]])

In [31]:
print(lsa_model.components_.shape)

(10, 5000)


In [32]:
terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74641), ('man', 0.45361), ('charge', 0.21087), ('new', 0.14088), ('court', 0.11163)]
Topic 2: [('man', 0.69428), ('charge', 0.30079), ('court', 0.16881), ('face', 0.11416), ('murder', 0.10649)]
Topic 3: [('new', 0.83622), ('plan', 0.23662), ('say', 0.18234), ('council', 0.11037), ('govt', 0.10891)]
Topic 4: [('say', 0.73993), ('plan', 0.35831), ('govt', 0.16982), ('council', 0.12798), ('fund', 0.07578)]
Topic 5: [('plan', 0.73543), ('council', 0.17432), ('govt', 0.14944), ('urge', 0.08), ('water', 0.06265)]
Topic 6: [('govt', 0.54878), ('urge', 0.25591), ('court', 0.24588), ('fund', 0.22754), ('face', 0.16043)]
Topic 7: [('charge', 0.51173), ('court', 0.4598), ('face', 0.34817), ('murder', 0.12791), ('plan', 0.11345)]
Topic 8: [('win', 0.64389), ('court', 0.30603), ('kill', 0.21496), ('australia', 0.09123), ('cup', 0.08607)]
Topic 9: [('win', 0.56214), ('charge', 0.4746), ('council', 0.12887), ('australia', 0.08343), ('open', 0.05495)]
Topic 10: [('council', 0.89

### LDA(Latent Dirichlet Allocation) : 토픽의 단어 분포, 즉 특정 토픽에 특정 단어가 나타날 확률을 추정합니다.

In [33]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


In [34]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.17024867, 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.0366096 , 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26687206, 0.02914502, 0.02914502, ..., 0.13007484, 0.02916018,
        0.28739608],
       [0.10378115, 0.02637829, 0.12325014, ..., 0.02637829, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.2255442 , ..., 0.03376055, 0.03376055,
        0.03376055]])

In [35]:
print(lda_model.components_.shape)

(10, 5000)


In [36]:
# LDA의 결과 토픽과 각 단어의 비중을 출력합시다
terms = tfidf_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(lda_model.components_, terms)

Topic 1: [('australia', 9359.06334), ('sydney', 5854.97288), ('attack', 4784.76322), ('change', 4193.63035), ('year', 3924.88997)]
Topic 2: [('government', 6344.07413), ('charge', 5947.12292), ('man', 4519.7974), ('state', 3658.16422), ('live', 3625.10473)]
Topic 3: [('australian', 7666.65651), ('say', 7561.01807), ('police', 5513.22932), ('home', 4048.38409), ('report', 3796.04446)]
Topic 4: [('melbourne', 5298.35047), ('south', 4844.59835), ('death', 4281.78433), ('china', 3214.44581), ('women', 3029.28443)]
Topic 5: [('win', 5704.0914), ('canberra', 4322.0963), ('die', 4025.63057), ('open', 3771.65243), ('warn', 3577.47151)]
Topic 6: [('court', 5246.3124), ('world', 4536.86331), ('country', 4166.34794), ('woman', 3983.97748), ('crash', 3793.50267)]
Topic 7: [('election', 5418.5038), ('adelaide', 4864.95604), ('house', 4478.6135), ('school', 3966.82676), ('2016', 3955.11155)]
Topic 8: [('trump', 8189.58575), ('new', 6625.2724), ('north', 3705.40987), ('rural', 3521.42659), ('donald',

#  비지도 학습 토크나이저

In [38]:
from konlpy.tag import Okt

kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


In [39]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워')) #미등록 단어를 기준으로 형태소를 분류해내는 것은 어렵다

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


In [40]:
#앞뒤로 등장하는 빈도를 고려하여 토큰화하는 방식이 필요 soynlp이것을 해결해 준다고 함

import urllib.request

txt_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

('/aiffel/aiffel/topic_modelling/data/2016-10-20.txt',
 <http.client.HTTPMessage at 0x7f134eaf0e20>)

In [41]:
from soynlp import DoublespaceLineCorpus

corpus = DoublespaceLineCorpus(txt_filename)
len(corpus)

30091

In [42]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

In [43]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 2.484 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [44]:
word_score_table["반포한"].cohesion_forward

0.08838002913645132

In [45]:
word_score_table["반포한강"].cohesion_forward

0.19841268168224552

In [46]:
word_score_table["반포한강공"].cohesion_forward

0.2972877884078849

In [47]:
word_score_table["반포한강공원"].cohesion_forward

0.37891487632839754

In [48]:
word_score_table["반포한강공원에"].cohesion_forward

0.33492963377557666

In [49]:
# branching_entropy 확인
word_score_table["디스"].right_branching_entropy

1.6371694761537934

In [50]:
word_score_table["디스플"].right_branching_entropy

-0.0

In [51]:
word_score_table["디스플레"].right_branching_entropy

-0.0

In [52]:
word_score_table["디스플레이"].right_branching_entropy

3.1400392861792916

In [53]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [54]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']